# 08 — Backtesting (R-multiples and portfolio summaries)

Backtesting validates the *rules* historically.

We measure in **R-multiples**:
- 1R is the distance from entry to stop
- if you win 2R, your profit equals 2× your risk
- it makes trades comparable across prices and sizes

This notebook shows:
- single ticker backtest
- multi-ticker portfolio backtest
- equity curve in R

In [1]:
# If running from repo root and editable install is not done:
# pip install -e ".[dev]"

import pandas as pd
pd.set_option("display.width", 140)
pd.set_option("display.max_columns", 50)

> Tip: many modules assume the benchmark **SPY** is present (for Relative Strength).
> When using real tickers, include SPY:
>
> `tickers = ["AAPL","MSFT","NVDA","SPY"]`

In [3]:
from swing_screener.data.market_data import fetch_ohlcv, MarketDataConfig
from swing_screener.backtest.simulator import BacktestConfig, backtest_single_ticker_R, summarize_trades
from swing_screener.backtest.portfolio import PortfolioBacktestConfig, backtest_portfolio_R, equity_curve_R

tickers = [
    "AAPL",
    "MSFT",
    "NVDA",
    "AMZN",
    "META",
    "GOOGL",
    "TSLA",
    "AMD",
    "NFLX",
    "AVGO",
    "INTC",
    "ORCL",
    "CRM",
    "QCOM",
    "ADBE",
    "CSCO",
    "SHOP",
    "UBER",
    "ABNB",
    "SPY",
]
ohlcv = fetch_ohlcv(tickers, MarketDataConfig(start="2018-01-01"))

bt_cfg = BacktestConfig(entry_type="pullback", pullback_ma=20, atr_window=14, k_atr=2.0, take_profit_R=2.0, max_holding_days=20)

trades_aapl = backtest_single_ticker_R(ohlcv, "AAPL", bt_cfg)
trades_aapl.head()

,ticker,entry_date,exit_date,entry,stop,tp,exit,exit_type,R,holding_days,entry_type
0,AAPL,2018-02-14,2018-03-06,39.3565,36.9476,44.1744,41.5434,time,0.9078,20,pullback
1,AAPL,2018-04-10,2018-04-24,40.7392,38.3544,45.5086,38.3544,stop,-1.0000,14,pullback
2,AAPL,2018-05-02,2018-05-22,41.5198,39.6360,45.2876,44.1798,time,1.4120,20,pullback
3,AAPL,2018-07-06,2018-07-26,44.3710,42.8137,47.4855,45.8439,time,0.9459,20,pullback
4,AAPL,2018-08-01,2018-08-16,47.5648,46.0628,50.5687,50.5687,take_profit,2.0000,15,pullback


In [4]:
summarize_trades(trades_aapl)

,trades,winrate,avg_R,median_R,expectancy_R,profit_factor_R
0,58,0.6379,0.4419,0.3788,0.4419,2.4518


In [5]:
trades_all, by_ticker, total = backtest_portfolio_R(ohlcv, ["AAPL","MSFT","NVDA"], PortfolioBacktestConfig(bt=bt_cfg, min_trades_per_ticker=1))
total

,trades,winrate,avg_R,median_R,expectancy_R,profit_factor_R
0,197,0.5381,0.2535,0.0947,0.2535,1.6867


In [6]:
curve = equity_curve_R(trades_all)
curve.tail()

,date,R,cum_R
180,2025-12-11,0.5492,51.7327
181,2025-12-17,-1.0000,50.7327
182,2025-12-29,-0.1815,50.5512
183,2026-01-08,0.3966,50.9478
184,2026-01-13,-1.0000,49.9478



## Details: backtest rules and exit priority
Single-ticker backtest (`backtest_single_ticker_R`):
- Entry happens at the close of the signal bar.
- Stop = entry - k * ATR; take profit = entry + take_profit_R * (entry - stop).
- Exit priority each day: stop -> take profit -> time exit.
- `min_history` skips tickers without enough bars.

Portfolio backtest (`backtest_portfolio_R`):
- Runs the single-ticker backtest per symbol.
- `min_trades_per_ticker` filters thin histories from the summary table.
- `equity_curve_R` sums R by exit date and cumulatively sums over time.



## Glossary (tickers and metrics)
- SPY: ETF tracking the S&P 500. We use it as the benchmark for Relative Strength (RS).
- SMA: Simple Moving Average. Used for trend filters and pullback signals.
- ATR: Average True Range. Measures typical daily range; used for stop distance and sizing.
- RS: Relative Strength = momentum vs benchmark (e.g., stock 6m return minus SPY 6m return).
- R (R-multiple): Risk unit where 1R = entry - stop. Used in backtests and trade management.
